In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import json

In [2]:
expressionDic = {
"0": "angry",
"1" : "disgust",
"2" : "fear",
"3" : "happy",
"4" : "sad",
"5" : "surprise",
"6" : "neutral"}

#incomplete
#https://github.com/tensorflow/tfjs-models/blob/838611c02f51159afdd77469ce67f0e26b7bbb23/face-landmarks-detection/src/mediapipe-facemesh/keypoints.ts
#https://github.com/tensorflow/tfjs-models/blob/646992fd7ab8237c0dc908f2526301414b417c95/face-landmarks-detection/mesh_map.jpg
facialLandmarks = {
  'silhouette': [
    10,  338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288,
    397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136,
    172, 58,  132, 93,  234, 127, 162, 21,  54,  103, 67,  109
  ],

  'lipsUpperOuter': [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291],
  'lipsLowerOuter': [146, 91, 181, 84, 17, 314, 405, 321, 375, 291],
  'lipsUpperInner': [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308],
  'lipsLowerInner': [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308],

  'rightEyeUpper0': [246, 161, 160, 159, 158, 157, 173],
  'rightEyeLower0': [33, 7, 163, 144, 145, 153, 154, 155, 133],
  'rightEyeUpper1': [247, 30, 29, 27, 28, 56, 190],
  'rightEyeLower1': [130, 25, 110, 24, 23, 22, 26, 112, 243],
  'rightEyeUpper2': [113, 225, 224, 223, 222, 221, 189],
  'rightEyeLower2': [226, 31, 228, 229, 230, 231, 232, 233, 244],
  'rightEyeLower3': [143, 111, 117, 118, 119, 120, 121, 128, 245],

  'rightEyebrowUpper': [156, 70, 63, 105, 66, 107, 55, 193],
  'rightEyebrowLower': [35, 124, 46, 53, 52, 65],

  'rightEyeIris': [473, 474, 475, 476, 477],

  'leftEyeUpper0': [466, 388, 387, 386, 385, 384, 398],
  'leftEyeLower0': [263, 249, 390, 373, 374, 380, 381, 382, 362],
  'leftEyeUpper1': [467, 260, 259, 257, 258, 286, 414],
  'leftEyeLower1': [359, 255, 339, 254, 253, 252, 256, 341, 463],
  'leftEyeUpper2': [342, 445, 444, 443, 442, 441, 413],
  'leftEyeLower2': [446, 261, 448, 449, 450, 451, 452, 453, 464],
  'leftEyeLower3': [372, 340, 346, 347, 348, 349, 350, 357, 465],

  'leftEyebrowUpper': [383, 300, 293, 334, 296, 336, 285, 417],
  'leftEyebrowLower': [265, 353, 276, 283, 282, 295],

  'leftEyeIris': [468, 469, 470, 471, 472],

  'midwayBetweenEyes': [168],

  'noseTip': [1],
  'noseBottom': [2],
  'noseRightCorner': [98],
  'noseLeftCorner': [327],

  'rightCheek': [205],
  'leftCheek': [425]
}

imageDir = r"C:\Users\myfir\My Drive\Skyline\ExpW\data\data\image\originPad\origin"
labelListPath = r"C:\Users\myfir\My Drive\Skyline\ExpW\data\data\label\label.lst"
headers = ["image_name", "face_id_in_image", "face_box_top" ,"face_box_left" ,"face_box_right" ,"face_box_bottom" ,"face_box_confidence" , "expression_label"]

data = pd.read_csv(labelListPath, delim_whitespace=True, header= None, names= headers, dtype= str)
data['expression_label'] = data['expression_label'].replace(expressionDic)
data= data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

data = data[['image_name', 'expression_label']]

In [3]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='face_landmarker.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)


In [4]:
#filenameList = data['image_name'].tolist()
#labelList = data['expression_label'].tolist()

filenamesList = [name for name in os.listdir(imageDir) if name.endswith('.jpg') and name in list(data['image_name'])]
validFilenames = []
expressionX = {expr: [[]] * 478 for expr in expressionDic.values()}
expressionY = {expr: [[]] * 478 for expr in expressionDic.values()}
expressionZ = {expr: [[]] * 478 for expr in expressionDic.values()}

for index, filename in enumerate(filenamesList):
    filepath = os.path.join(imageDir, filename)
    # STEP 3: Load the input image.
    image = mp.Image.create_from_file(filepath)

    # STEP 4: Detect face landmarks from the input image.
    detection_result = detector.detect(image)

    #print(len(detection_result.face_landmarks))
    if len(detection_result.face_landmarks) == 1:
        validFilenames.append(filename)
        for face in detection_result.face_landmarks:
            for id, landmark in enumerate(face):
                x = landmark.x
                y = landmark.y
                z = landmark.z
                rowIndex = data[data['image_name'] == filename].index[0]
                expr = data.at[rowIndex, 'expression_label']

                expressionX[expr][id].append(x)
                expressionY[expr][id].append(y)
                expressionZ[expr][id].append(z)
    if (index+1) % 500 == 0:
        print(f'Processed {index} images...')
        outputFile = f'combined_XYZ_{index}.json'
        combineXYZ = {'X' : expressionX, 'Y': expressionY, 'Z': expressionZ}
        with open(outputFile, 'w') as out:
            json.dump(combineXYZ, out, indent= 4)
        
        expressionX = {expr: [[]] * 478 for expr in expressionDic.values()}
        expressionY = {expr: [[]] * 478 for expr in expressionDic.values()}
        expressionZ = {expr: [[]] * 478 for expr in expressionDic.values()}
        combineXYZ = {}

outputFile = f'combined_XYZ_{index}.json'
combineXYZ = combineXYZ = {'X' : expressionX, 'Y': expressionY, 'Z': expressionZ}
with open(outputFile, 'w') as out:
    json.dump(combineXYZ, out, indent= 4)



c:\Users\myfir\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processed 499 images...
Processed 999 images...
Processed 1499 images...
Processed 1999 images...
Processed 2499 images...
Processed 2999 images...
Processed 3499 images...
Processed 3999 images...
Processed 4499 images...
Processed 4999 images...
Processed 5499 images...
Processed 5999 images...
Processed 6499 images...
Processed 6999 images...
Processed 7499 images...
Processed 7999 images...
Processed 8499 images...
Processed 8999 images...
Processed 9499 images...
Processed 9999 images...
Processed 10499 images...
Processed 10999 images...
Processed 11499 images...
Processed 11999 images...
Processed 12499 images...
Processed 12999 images...
Processed 13499 images...
Processed 13999 images...
Processed 14499 images...
Processed 14999 images...
Processed 15499 images...
Processed 15999 images...
Processed 16499 images...
Processed 16999 images...
Processed 17499 images...
Processed 17999 images...
Processed 18499 images...
Processed 18999 images...
Processed 19499 images...
Processe

OSError: [Errno 28] No space left on device

In [ ]:
len(filenamesList)

18642